In [1]:

def odd_int_check(x):
    # check if given input is an odd integer, if not return an error
    if x % 2 != 0 and isinstance(x,int):
        pass
    else:
        raise ValueError(f'The number given {x} must be an odd integer.')




def count_r_steps(q, n):
    odd_int_check(q) #check if variable is an odd integer
    odd_int_check(n) #check if variable is an odd integer

    r_count = 0
    n = q * n + 1 #perform odd operation
        
    # perform even operations and keep count
    while n % 2 == 0: 
        n /= 2
        r_count += 1
        
    return r_count





def write_r_families(q, int_i, int_f, r_cutoff=float('inf')):

    odd_int_check(int_i) #check if variable is an odd integer

    r_dict = {}

    # create list of odd integers to get r-steps
    for n in range(int_i, int_f, 2):
        r_count = count_r_steps(q, n) # get r vale

        # Check if the key exists in r_families, and create it if it doesn't
        if r_count not in r_dict:
            r_dict[r_count] = []

        # Only add the odd to its r-family if there is space (i.e. r-family is below cutoff length)
        if len(r_dict[r_count]) < r_cutoff:
            r_dict[r_count].append(n)

    # Sort the r-families by increasing r-value
    sorted_r_dict = dict(sorted(r_dict.items()))

    sorted_r_dict['q'] = q

    return sorted_r_dict






import csv

def r_dict_to_csv(r_dict):
    # Create a CSV file and write the data
    with open('output.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['q', r_dict['q']])
        writer.writerow(['len', 'r', 'odds'])
        for key, values in r_dict.items():

            if isinstance(key, int):
                writer.writerow([len(values), key] + values)







def complete_r_family(q:int, n_i:int, r_len_cutoff:int=10):

    r_family = [n_i]

    odd_int_check(n_i) #check if variable is an odd integer
    r_count_i = count_r_steps(q, n_i) # get r vale

    n = n_i
    while len(r_family) < r_len_cutoff:

        if r_family[0] - 2**(r_count_i+1) > 0:
            n -= 2**(r_count_i+1)
        else:
            n = r_family[-1] + 2**(r_count_i+1)

        r_count = count_r_steps(q, n)

        if r_count != r_count_i:
            raise ValueError(f'The resulting odd {n}')
        
        r_family.append(n)
        r_family.sort()
    
    if r_family[0] - 2**(r_count_i+1) > 0:
        print(f'The smallest odd integer for r={r_count_i} was not found. Try a smaller initial odd ({n_i}) or a larger cutoff length ({r_len_cutoff}).')

    return r_count, r_family




def make_modulo_r_family(r_count:int, r_family:list):

    differences = []

    for i in range(len(r_family) - 1):
        diff = r_family[i + 1] - r_family[i]
        differences.append(diff)

    # Check if all elements in the list are the same
    all_elements_same = all(x == differences[0] for x in differences)

    if not all_elements_same:
        raise ValueError("Not all elements in 'differences' are the same.")

    remainder = differences[0]

    if r_family[0] - remainder > 0:
        print(f'The smallest odd integer for the r={r_count} family was not found. Try reloading your r-family to include an integer such that {r_family[0]} < {remainder}.')

    smallest_odd = r_family[0]

    return (smallest_odd, remainder)



def write_r_families(q:int, r_count_i:int, r_count_f:int, r_len_cutoff:int = 20):

    r_dict = {}
    n = 1

    while set(range(r_count_i, r_count_f + 1)) - set(r_dict.keys()):

        skip_r_family_build = False #reset skip block boolean



        for r_key in r_dict:
            r_family_dict = r_dict[r_key]
            r_family = r_family_dict['r_family']
            r_modulo = r_family_dict['modulo']

            odd_in_r_family_check = check_in_modulo_family(n, r_modulo)

            if odd_in_r_family_check:
                skip_r_family_build = True  # Set the flag to skip the code block
                break  # Exit the for loop


        if not skip_r_family_build:
            r_count, r_family = complete_r_family(q, n, r_len_cutoff)

            if (r_count >= r_count_i) and (r_count <= r_count_f):
                modulo = make_modulo_r_family(r_count, r_family)
                r_dict[r_count] = {'modulo': modulo, 'r_family': r_family}

                # Sort the r-families by increasing r-value
                r_dict = dict(sorted(r_dict.items()))

        n += 2

    return r_dict



def check_in_modulo_family(n_check:int, modulo_tuple:tuple):

    smallest_odd, remainder = modulo_tuple

    if (n_check - smallest_odd) % remainder == 0:
        in_r_family = True
    else:
        in_r_family = False

    return in_r_family





In [12]:
q = 7
r_i = 1
r_f = 23

r_dict = write_r_families(q, r_i, r_f, r_len_cutoff = 10)

for keys in r_dict.keys():
    print(keys, r_dict[keys]['modulo'])

# print(r_dict)

1 (3, 4)
2 (5, 8)
3 (1, 16)
4 (25, 32)
5 (41, 64)
6 (9, 128)
7 (201, 256)
8 (329, 512)
9 (73, 1024)
10 (1609, 2048)
11 (2633, 4096)
12 (585, 8192)
13 (12873, 16384)
14 (21065, 32768)
15 (4681, 65536)
16 (102985, 131072)
17 (168521, 262144)
18 (37449, 524288)
19 (823881, 1048576)
20 (1348169, 2097152)
21 (299593, 4194304)
22 (6591049, 8388608)
23 (10785353, 16777216)


In [4]:
# q = 1
# n_i = 5


# for n_i in range(1,29,2):

#     r, r_family = complete_r_family(q, n_i, 10)
#     print(n_i, r, r_family)


# odd, rem = make_modulo_r_family(r, r_family)

# # Define the integer to check
# int_2_check = n_i + 8
# print(int_2_check)

# check_in_modulo_family(int_2_check, odd, rem)


In [ ]:
r_count_i = 2
r_count_f = 3

r_dict = {2: 'a', 3: 2}

print(range(r_count_i, r_count_f))
print(range(r_count_i, r_count_f + 1))

print(set(range(r_count_i, r_count_f + 1)))

print(set(r_dict.keys()))

print()

print(set(range(r_count_i, r_count_f + 1)) - set(r_dict.keys()))
print(bool(set(range(r_count_i, r_count_f + 1)) - set(r_dict.keys())))

range(2, 3)
range(2, 4)
{2, 3}
{2, 3}

set()
False


In [ ]:
# # count_r_steps(5, 7)
# r_dict = write_r_families(5, 1, 800000, 20)


# for k in r_dict.keys():
#     print(k, r_dict[k])


# r_dict_to_csv(r_dict)


